# Gap Analysis Project


### The objective of this project is:
+ Download 5 minutes historical quotes for NYSE stocks
+ 5 Mts data includes(**CVS** format):
    + Ticker, Date, time, open, high, low, close, volume
+ Download gapped up and gapped down stocks i.e., Gap List(in **CVS** format)
+ Upload 5 mts quotes for all the stocks in Gap List to a MS Access Database
+ Filter the records in include only quote data for the following specific 
  times:
    +    9:35 AM
    +    10:00 AM
    +    10:30 AM
    +    11:00 AM
    +    4:00 PM
+ The goal is to see how quickly the gap, fades or continues
+ Next step is to transpose this data from multiple records for the 
  specific times stated above into a single record for a day
      
 
    

** Example of original file **

![](selected_time_quotes1.png)

** Transposed File Data: ** 
In a single record, it includes all 6 time period data

![](transpose_desc.png)

** Sample transposed record layout **

![](transposed.png)

+ Using the transposed record quotes, create a **candle chart** and 
  display it using a **GUI (Tkinter)** front end for a given day

Some of the **highlights** on technical front/design:
+ Follow **MVC** model
+ **Eliminate repetitive** coding
+ Follow **Modular** coding
+ **Automate Database scripts** as much as possible 

## Historical 5 Mts Data

This data was downloaded from [stooq site](https://stooq.com/db/h/)

This is the directory on C: drive where the downloaded CSV files are stored

### Directory listing


In [5]:
import os
import os.path

%ls *.txt


 Volume in drive C is OS
 Volume Serial Number is 2E74-9FAC

 Directory of C:\GapAnalysisNB-GITHub

04/01/2018  03:32 AM            49,219 a.us.txt
04/01/2018  03:32 AM            49,892 aa.us.txt
04/01/2018  03:32 AM            51,575 aac.us.txt
04/01/2018  03:32 AM            49,307 aan.us.txt
04/01/2018  03:32 AM            52,568 aap.us.txt
04/01/2018  03:32 AM            48,236 aat.us.txt
               6 File(s)        300,797 bytes
               0 Dir(s)  410,108,526,592 bytes free


There are **3802** files in this directory corresponding to 3802 ticker
symbols.

### Contents of a 5 Mts Ticker File ('A')

In [6]:
import csv
f = open("a.us.txt")
recs = csv.reader(f)
for rec in recs:
    print(rec)
f.close()

['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'OpenInt']
['2018-03-14', '14:35:00', '71.07', '71.16', '70.75', '71.01', '26350', '0']
['2018-03-14', '14:40:00', '70.97', '71', '70.72', '70.9168', '11834', '0']
['2018-03-14', '14:45:00', '70.95', '70.98', '70.8', '70.82', '7978', '0']
['2018-03-14', '14:50:00', '70.78', '70.92', '70.78', '70.85', '8810', '0']
['2018-03-14', '14:55:00', '70.84', '70.93', '70.84', '70.91', '3580', '0']
['2018-03-14', '15:00:00', '70.89', '70.91', '70.72', '70.74', '5100', '0']
['2018-03-14', '15:05:00', '70.76', '70.91', '70.76', '70.8725', '4578', '0']
['2018-03-14', '15:10:00', '70.85', '70.95', '70.83', '70.95', '5129', '0']
['2018-03-14', '15:15:00', '70.93', '70.93', '70.68', '70.68', '6386', '0']
['2018-03-14', '15:20:00', '70.72', '70.77', '70.66', '70.69', '6500', '0']
['2018-03-14', '15:25:00', '70.685', '70.81', '70.685', '70.79', '2200', '0']
['2018-03-14', '15:30:00', '70.82', '70.86', '70.82', '70.86', '1600', '0']
['2018-03-14',

## Gap List File

This file is downloaded from [Finviz](https://elite.finviz.com/screener.ashx?v=171&f=sh_avgvol_o500,sh_curvol_o750,sh_price_o2,sh_relvol_o1.5,ta_gap_u1&ft=4&o=-volume)

These stocks are Gapped up or down by recently by 0.5% of theie price and
with significant volume (over 500,000 shares). We will do gap analysis only for these stocks. We will pull 5 Mts data for only those stickers on the Gap List as the Gap Analysis Base. 


### Sample Gap file contents ###

![](gap_file.png)

![](gap_list.png)

There are **987** rows corresponding to 987 ticker symbols we are interested
for Gap Analysis research

The program reads both 5 Mts File and Gap List files and appends all the 5 Mte
quote records for all the gapped tickers and appends to an existing MS Access 
table called tbl5MtsRawFile using pyodbc library.

There are 2 python programs does this job

** 1. procAllFiles **

This program reads Gap List files and calls proc1file, passing a specific 
ticker symbol. It does this for all 987 tickers

In [7]:
'''
Created on Jun 4, 2018

@author: karsu
'''
#z tbl5MtsRawFile
import os
#import pyodbc
import proc1file
import csv
#output = 'csv'
output = 'db'


    
#dirname = "C:\\TickData2018\\nysestocks_temp\\"
dirname = ""
   
if (output == 'csv'):
    f1 = open('5MtsFile.csv', 'w')
    hdrData = ["Ticker", "CurDate", "CurTime",
           "Open", "Low", "High", "Close", "Volume"]   
    myFile = open('5MtsFile.csv', 'w', newline='')
    with myFile:
        writer = csv.writer(myFile)
        writer.writerow(hdrData)
        
# if this CSV file has to be imported into MS Access DB, you have to first save this as MS Excel Book in Excel
 
fileNum = 0

selected_tickers_file = "selectedTickers.csv"

selectedTickers = csv.reader(open(selected_tickers_file, 'r'), delimiter=',')

for item in selectedTickers:
#    print(item)
    ticker_name = item[1]
    print(ticker_name)
#        for x in os.listdir(dirname):
    fname = dirname + ticker_name + ".us.txt"
    if (os.path.isfile(fname)) :
        fileNum = fileNum+1
    #    if (fileNum == 2):
    #        break
#        print(fname)
        proc1file.getSingleFileData(fname,output)
print('Program Completed')

Ticker
A
AA
AABA
AAL
AAN
AAOI
AAXJ
AAXN
ABBV
ABC
ABEO
ABM
ABT
ACAD
ACC
ACGL
ACHC
ACIA
ACIW
ACM
ACWI
ADI
ADM
AEE
AEL
AEM
AEO
AEP
AER
AFL
AGCO
AGO
AIG
AIV
AIZ
AJG
AJRD
AKAM
AKAO
AKR
AKRX
ALB
ALDR
ALK
ALL
ALLE
ALLY
ALSN
AMAT
AMBA
AMC
AMCX
AME
AMTD
AN
ANDX
ANF
AOS
APA
APC
APH
APO
APTV
APY
AQUA
AR
ARMK
ARNA
ARNC
ARRS
ARRY
ARWR
ASB
ATH
ATI
ATKR
ATUS
ATVI
AVT
AVYA
AWK
AXP
AXTA
AYX
AZN
AZUL
BAC
BAH
BAS
BAX
BBL
BBT
BBY
BC
BDC
BECN
BEN
BERY
BF-B
BG
BGS
BHF
BHGE
BHP
BIG
BILI
BITA
BK
BLDR
BLL
BLMN
BLNK
BMRN
BMY
BNS
BOFI
BOOT
BOX
BP
BPL
BRKS
BRS
BRZU
BSX
BTI
BTU
BUD
BWA
BX
BXS
BYD
BZUN
C
CA
CADE
CAG
CAH
CAR
CARA
CARG
CARS
CASY
CATM
CAVM
CBAY
CBOE
CBRE
CBS
CC
CCE
CCK
CCL
CDEV
CDK
CDNS
CDW
CELG
CENX
CERN
CF
CFG
CFX
CGNX
CHD
CHGG
CHKP
CHRS
CHRW
CHS
CIEN
CINF
CIT
CJ
CL
CLR
CLSD
CMA
CMC
CMCM
CMCSA
CMS
CNI
CNK
CNQ
CNX
COF
COG
COLD
COMM
CONE
CONN
COP
CORT
COUP
CPB
CPE
CPRT
CPT
CRC
CREE
CRH
CRK
CROX
CRSP
CRTO
CRUS
CRZO
CSCO
CSFL
CSIQ
CSOD
CSX
CTB
CTRP
CTSH
CVLT
CVNA
CVRR
CVS
CWH
D
DAL
DAN
DATA
DBD
DBX
DCT


As you can see that not all the tickers from Gapped Up list don't have corresponding records in 5 Mts file. Some of them are  from NASDAQ. Initially, the focus was on NYSE stocks.

In [ ]:
'''
Created on Jun 4, 2018

@author: karsu
'''
#import os, sys
#import pyodbc
import csv
import db_utils as dbu
import os

def getSingleFileData(filename, output):
 
    rawFile = filename
    removePath = rawFile.split("\\") [-1]
    ticker = removePath.split('.')[0]    # format - Date, Time, open, high, low, close, volume
    # print(data.read())
    
    temp_line = []
    total_result = []
    single_result = []
    lineNo = 1
    hoursadj = 0

    data = open(rawFile, 'r')
    for line in data:
        if (lineNo == 1):
            lineNo = lineNo + 1
            continue
        temp_line = line.split(',')

        if (lineNo == 2):
            olddate = temp_line[0]
            lineNo = 0

        timelist = temp_line[1].split(":")
        hours = int(timelist[0])
        

        if (olddate != temp_line[0]):
            hoursadj = hours - 14
            olddate = temp_line[0]
              
        minutes = int(timelist[1])
        seconds = int(timelist[2])
        hours = hours - 5 - hoursadj
        if (hours >= 12):
            amorpm = "PM"
        else:
            amorpm = "AM"
            
        if (hours > 12):
            hours = hours-12
                
        trtime = str(hours) + ":" + str(format(minutes, '02d')) + ":"+ str(format(seconds, '02d'))+ " " + amorpm        
        single_result = []
        single_result.insert(0, ticker)
        single_result.insert(1, temp_line[0])
        
        single_result.insert(2, trtime)

        single_result.insert(3, temp_line[2])
        single_result.insert(4, temp_line[4])
        single_result.insert(5, temp_line[3])
        single_result.insert(6, temp_line[5])
        single_result.insert(7, temp_line[6])
        total_result.append(single_result)
    
    if (output == 'csv'):
        myFile = open('5MtsFile.csv', 'a', newline='')
        with myFile:
            writer = csv.writer(myFile)
            writer.writerows(total_result)
    elif (output == 'db'):
#        db_file = r'''C:\TickData2018\StooqData.accdb'''  #raw string, escape sequences are ignored
        db_file = 'StooqData.accdb'  #raw string, escape sequences are ignored
#        if (os.path.isfile(db_file)) :
#            print('file present')
        db_file = os.path.abspath(db_file)
        conn = dbu.createDBConnection(db_file)
        c = conn.cursor()
        for row in total_result:
            sql = """INSERT INTO tbl5MtsRawFile VALUES ('%s', '%s', '%s','%s', '%s', '%s', '%s', '%s')""" \
            % (row[0], row[1], row[2],row[3], row[4], row[5], row[6], row[7])            
            c.execute(sql)
            conn.commit()
    else:
        print('output argument has to be either csv or db')

    if (output == 'db'):
        c.close()
        del c
        conn.close()
                
#    print("Program Completed")

The tbl5MtsRawFile table in StooqData database is scrubbed for missing critical time slots that 
Gap Analysis depends on such as 9:35 AM and 4:00 PM. There are many MS Access DB queries are written 
and executed within MS Access front-end to scrub the data.

![](stooq_raw_DB.png)

Once the **tbl5MtsRawFile** table is scrubbed, a new MS Access Database called **StooqDataAnalysis** 
is created to which tables from StooqData Database is attched to.  
  
This is considered as a best practice to segregate raw data processing DB from analysis DB for several 
reasons, i.e, smaller footprint for analysis database, inadvertent changes to raw tables, added security etc.

In [13]:
%cd c:/TickData2018/
%ls stoo*

c:\TickData2018
 Volume in drive C is OS
 Volume Serial Number is 2E74-9FAC

 Directory of c:\TickData2018

06/18/2018  08:24 AM       232,427,520 stooqData.accdb
06/18/2018  08:24 AM               192 stooqData.laccdb
06/18/2018  08:24 AM        50,466,816 StooqDataAnalysis.accdb
               3 File(s)    282,894,528 bytes
               0 Dir(s)  410,405,457,920 bytes free


![](stooq_analysis_DB.png)

From now on we will be working with **StooqDataAnalysis** database.

The raw 5 mts table contains the quote data for 5 mts tick data. For our Gap Analysis we only need dat for:
+ 9:35 AM
+ 10:00 AM
+ 10:30 AM
+ 11:00 AM
+ 4:00 PM

The first step is to extract only these records through a MS Access Query.

The raw 5 mts table contains 539,466 records.

![](base_table_rec_count.png)

The extracted data is stored in a table called **5MtsBaseForAnalysis**  
This table will form the base for our Gap Analysis.

**5MtsBaseForAnalysis** contains 34,598 records, after the extraction.

![](extracted_rec_count.png)

For doing gap analysis, the 2 most important tick data we need are:
+ previous days 'close' tick data (4:00 PM data)
+ current day's 'open' tick data (9:35 AM data) 

![](selected_time_quotes1.png)

It will be easy for the analysis, if we can get both these tick data on a single record,  
i.e., 3/14's 4:00 PM data and 3/15's 9:35 AM data

To find out whether the gap trend continued or faded in the morning session, we need to analyse tick data  
for 10:00 AM, 10:30 AM, 11:00 AM.  
  
3/15's 4:00 PM tick data is required for next day (3/16) as previous close data.  
  
So, our goal is to capture all these tick data from 6 records into a one single record for easier analysis.

**Transposed File Data**  
![](transpose_desc.png)

** Sample transposed record layout **  
![](transposed.png)

The following python program, **createStooqCrossTabTable**, creates the transposed table. For all it's database transactions/activities,  
it calls **data objects** (database io objects) corresponding to their tables.  

+ **io_5MtsBaseForAnalysisSorted**
+ **io_StooqCrossTabTable**

It also uses a **common shared database utility** (python project/library) stored outside of the Gap Analysis  
project/directory. So this can be shared by all python projects. It is stored at:  

**C:\myPythonLibs\myDBUtils\db_utils.py**

    

 ## createStooqCrossTabTable ##

In addition to creating transposed record, additional features include:
+ a beginning date is specified from when the transposed records to be created, this is implemented  
  as the entire download process will be run periodically when historical data available on  
  the download site
+ the program skips transpose process for the first day of all the tickers as there is no  
  previous day data readily availble, so skips all tick data records except 4:00 PM tick record for 
  sourcing the prev close tick data for the following day

In [3]:
'''
Program  Name - createStooqCrossTabTable

Created on Jun 10, 2018
@author: karsu
'''
#import makeAccessDBConnection
import os, sys
import pyodbc
import io_5MtsBaseForAnalysisSorted as base5
import io_StooqCrossTabTable as scrt
import datetime
import logging
import db_utils as dbu

logging.basicConfig(filename='test.log', level=logging.DEBUG, format = '%(module)s:%(levelname)s:%(lineno)d:%(message)s')
    
def move_prev_close_data_to_outrec():
    outrec['PDayCloseTime'] = prevclosedata['CDay0400PMTime']
    outrec['PDayOpen'] = prevclosedata['CDay0400PMOpen']
    outrec['PDayLow'] = prevclosedata['CDay0400PMLow']
    outrec['PDayHigh'] = prevclosedata['CDay0400PMHigh']
    outrec['PDayClose'] = prevclosedata['CDay0400PMClose']
    outrec['PDayVolume'] = prevclosedata['CDay0400PMVolume']

def setPrevDayData():
    prevclosedata['Ticker'] =  inrec['Ticker']
    prevclosedata['CurDate'] = inrec['CurDate']
    prevclosedata['CDay0400PMTime'] = inrec['CurTime']
    prevclosedata['CDay0400PMOpen'] = inrec['Open']
    prevclosedata['CDay0400PMLow'] = inrec['Low']
    prevclosedata['CDay0400PMHigh'] = inrec['High']
    prevclosedata['CDay0400PMClose'] = inrec['Close']
    prevclosedata['CDay0400PMVolume'] = inrec['Volume']

def setnine35amData():
    outrec['Ticker'] =  inrec['Ticker']
    outrec['CurDate'] = inrec['CurDate']
    outrec['CDayOpenTime'] = inrec['CurTime']
    outrec['CDayOpenOpen'] = inrec['Open']
    outrec['CDayOpenLow'] = inrec['Low']
    outrec['CDayOpenHigh'] = inrec['High']
    outrec['CDayOpenClose'] = inrec['Close']
    outrec['CDayOpenVolume'] = inrec['Volume']

def settenamData():
    outrec['CDay1000AMTime'] = inrec['CurTime']
    outrec['CDay1000AMOpen'] = inrec['Open']
    outrec['CDay1000AMLow'] = inrec['Low']
    outrec['CDay1000AMHigh'] = inrec['High']
    outrec['CDay1000AMClose'] = inrec['Close']
    outrec['CDay1000AMVolume'] = inrec['Volume']

def setten30amData():
    outrec['CDay1030AMTime'] = inrec['CurTime']
    outrec['CDay1030AMOpen'] = inrec['Open']
    outrec['CDay1030AMLow'] = inrec['Low']
    outrec['CDay1030AMHigh'] = inrec['High']
    outrec['CDay1030AMClose'] = inrec['Close']
    outrec['CDay1030AMVolume'] = inrec['Volume']

def setelevenamData():
    outrec['CDay1100AMTime'] = inrec['CurTime']
    outrec['CDay1100AMOpen'] = inrec['Open']
    outrec['CDay1100AMLow'] = inrec['Low']
    outrec['CDay1100AMHigh'] = inrec['High']
    outrec['CDay1100AMClose'] = inrec['Close']
    outrec['CDay1100AMVolume'] = inrec['Volume']

def setfourpmData():
    outrec['CDay0400PMTime'] = inrec['CurTime']
    outrec['CDay0400PMOpen'] = inrec['Open']
    outrec['CDay0400PMLow'] = inrec['Low']
    outrec['CDay0400PMHigh'] = inrec['High']
    outrec['CDay0400PMClose'] = inrec['Close']
    outrec['CDay0400PMVolume'] = inrec['Volume']
    

#===============================================================================

def initprevclosedata():
#    logging.debug('prev close data initialized')
    prevclosedata = {
        'Ticker': None,
        'CurDate':None,
        'CDay0400PMTime':None,
        'CDay0400PMOpen':None,
        'CDay0400PMLow':None,
        'CDay0400PMHigh':None,
        'CDay0400PMClose':None,
        'CDay0400PMVolume':None    
    }
    return prevclosedata

def initinrecdata():
#    logging.debug('in-rec initialized')
    inrec = {}
    for column in baseColumns:
        inrec[column[0]] = None
    return inrec

def initoutrecdata():
    outrec = {}
    for column in outrecColumns:
        outrec[column[0]] = None
    return outrec
#===============================================================================

db_file = 'StooqDataAnalysis.accdb'  #raw string, escape sequences are ignored
db_file = os.path.abspath(db_file)
    
conn = dbu.createDBConnection(db_file)

c = conn.cursor()

begdate = datetime.date(2018, 3, 14)

fourpmtime = datetime.time(16, 0, 0)

nine35amtime = datetime.time(9, 35, 0)

tenamtime = datetime.time(10, 0, 0)

ten30amtime = datetime.time(10, 30, 0)

elevenamtime = datetime.time(11, 0, 0)


baseColumns = base5.get5MtsBaseForAnalysisSortedColumnsDetails(conn)
outrecColumns = scrt.getCrossTabColumnsDetails(conn)

inrecsall = base5.get_all_5MtsBaseForAnalysisSorted_recs(conn)
#logging.debug('total in-recs = ' + str(len(inrecsall)))

if (len(inrecsall)) <= 0 :
    print('no records in 5MtsBaseForAnalysisSorted table')
    sys.exit()

firstday = True
firstTickerrec = True

pticker = inrecsall[0]['Ticker']
pdate = inrecsall[0]['CurDate'].date()

inrec = {}
prevclosedata = {}
outrec = {}

inrec = initinrecdata()
outrec = initoutrecdata()
prevclosedata = initprevclosedata()

#print(prevclosedata)
#print(prevclosedata['CDay0400PMTime'])

for rec in inrecsall:
    inrec = initinrecdata()

    inrec['Ticker'] = rec['Ticker']
    inrec['CurDate'] = rec['CurDate']
    inrec['CurTime'] = rec['CurTime']
    inrec['Open'] = rec['Open']
    inrec['Low'] = rec['Low']
    inrec['High'] = rec['High']
    inrec['Close'] = rec['Close']
    inrec['Volume'] = rec['Volume']

    cticker = inrec['Ticker']
    cdate = inrec['CurDate'].date()
    ctime = inrec['CurTime'].time()
    
#    logging.debug('Prev Ticker,date = {} - {} - {}'.format(pticker,pdate))
#    logging.debug('curr Ticker,date,time = {} - {} - {}'.format(cticker,cdate,ctime))
#    print('previous : ' + pticker + ' ' + str(pdate))
#    print('Current : ' + cticker + ' ' + str(cdate) + ' ' + str(ctime))
           
    if (pticker != cticker):
#        write_to_db(outrec)
        scrt.insert_recs(conn, outrec)        
        pticker = inrec['Ticker']
        pdate = inrec['CurDate'].date()
#        ptime = inrec['CurTime'].time()
        firstday = True
        outrec = initoutrecdata()
    elif (pdate != cdate):
        if (outrec['PDayCloseTime']):
#            write_to_db(outrec)
            scrt.insert_recs(conn, outrec)        
            pdate = inrec['CurDate'].date()
            outrec = initoutrecdata()
        else:
            pdate = inrec['CurDate'].date()            
        
    if (firstTickerrec):
        if (cdate < begdate):
            continue
        else:
            pticker = inrec['Ticker']
            pdate = inrec['CurDate'].date()
#            ptime = inrec['CurTime'].time()
            firstTickerrec = False
                
    if (firstday):
        if (ctime != fourpmtime) :
            continue
        else:
            setPrevDayData()
            firstday = False
            continue

    if  (ctime == nine35amtime) :
        move_prev_close_data_to_outrec()
        prevclosedata = initprevclosedata()
        setnine35amData()
#        print(outrec)
        continue
   
    if  (ctime == tenamtime) :
        settenamData()
        continue
   
    if  (ctime == ten30amtime) :
        setten30amData()
        continue
   
    if  (ctime == elevenamtime) :
        setelevenamData()
        continue
   
    if  (ctime == fourpmtime) :
        setfourpmData()
        setPrevDayData()
        continue
   
print('Program Completed')  

Program Completed


## io_5MtsBaseForAnalysisSorted ##

In [ ]:
'''

Program Name = io_5MtsBaseForAnalysisSorted

Created on Jun 10, 2018

@author: karsu
'''
import pyodbc
import datetime
import db_utils as dbu
import os

def get5MtsBaseForAnalysisSortedColumnsDetails(conn):
    return dbu.getTableColumnsDetails(conn, '5MtsBaseForAnalysisSorted')



def get_all_5MtsBaseForAnalysisSorted_recs(conn):
    recs = dbu.get_all_recs(conn, '5MtsBaseForAnalysisSorted')
    return recs

if __name__ == '__main__': 
    db_file = r'''StooqDataAnalysis.accdb'''  #raw string, escape sequences are ignored
    db_file = os.path.abspath(db_file)
    dbConnData = dbu.createDBConnection(db_file)
    conn = dbu.createDBConnection(db_file)   

## io_StooqCrossTabTable ##

In [ ]:
'''
Program Name = io_StooqCrossTabTable

Created on Jun 10, 2018

@author: karsu
'''

import pyodbc
import datetime
import makeAccessDBConnection
import db_utils as dbu
import os


#===============================================================================

def insert_recs(conn, outrec):
    
    c = conn.cursor()
    

    sql = """ insert into StooqCrossTabTable (Ticker, \
    CurDate, \
    PDayCloseTime, \
    PDayOpen, \
    PDayLow, \
    PDayHigh, \
    PDayClose, \
    PDayVolume, \
    CDayOpenTime, \
    CDayOpenOpen, \
    CDayOpenLow, \
    CDayOpenHigh, \
    CDayOpenClose, \
    CDayOpenVolume, \
    CDay1000AMTime, \
    CDay1000AMOpen, \
    CDay1000AMLow, \
    CDay1000AMHigh, \
    CDay1000AMClose, \
    CDay1000AMVolume, \
    CDay1030AMTime, \
    CDay1030AMOpen, \
    CDay1030AMLow, \
    CDay1030AMHigh, \
    CDay1030AMClose, \
    CDay1030AMVolume, \
    CDay1100AMTime, \
    CDay1100AMOpen, \
    CDay1100AMLow, \
    CDay1100AMHigh, \
    CDay1100AMClose, \
    CDay1100AMVolume, \
    CDay0400PMTime, \
    CDay0400PMOpen, \
    CDay0400PMLow, \
    CDay0400PMHigh, \
    CDay0400PMClose, \
    CDay0400PMVolume    
    ) values \
    (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) """
    
    c.execute(sql, \
    outrec['Ticker'], \
    outrec['CurDate'], \
    outrec['PDayCloseTime'], \
    outrec['PDayOpen'], \
    outrec['PDayLow'], \
    outrec['PDayHigh'], \
    outrec['PDayClose'], \
    outrec['PDayVolume'], \
    outrec['CDayOpenTime'], \
    outrec['CDayOpenOpen'], \
    outrec['CDayOpenLow'], \
    outrec['CDayOpenHigh'], \
    outrec['CDayOpenClose'], \
    outrec['CDayOpenVolume'], \
    outrec['CDay1000AMTime'], \
    outrec['CDay1000AMOpen'], \
    outrec['CDay1000AMLow'], \
    outrec['CDay1000AMHigh'], \
    outrec['CDay1000AMClose'], \
    outrec['CDay1000AMVolume'], \
    outrec['CDay1030AMTime'], \
    outrec['CDay1030AMOpen'], \
    outrec['CDay1030AMLow'], \
    outrec['CDay1030AMHigh'], \
    outrec['CDay1030AMClose'], \
    outrec['CDay1030AMVolume'], \
    outrec['CDay1100AMTime'], \
    outrec['CDay1100AMOpen'], \
    outrec['CDay1100AMLow'], \
    outrec['CDay1100AMHigh'], \
    outrec['CDay1100AMClose'], \
    outrec['CDay1100AMVolume'], \
    outrec['CDay0400PMTime'], \
    outrec['CDay0400PMOpen'], \
    outrec['CDay0400PMLow'], \
    outrec['CDay0400PMHigh'], \
    outrec['CDay0400PMClose'], \
    outrec['CDay0400PMVolume'])
    
    conn.commit()
    
def getCrossTabColumnsDetails(conn):
    return dbu.getTableColumnsDetails(conn, 'StooqCrossTabTable')

def get_all_StooqCrossTabTable_recs(conn):
    recs = dbu.get_all_recs(conn, 'StooqCrossTabTable')
    return recs


def getFieldType(field_names,column_details):
    for rec in column_details:
        if (rec[0] == field_names):
            return rec[1]
    
def get_selective_recs(conn,field_names,operands, values):

    column_details = getCrossTabColumnsDetails(conn)
    critera_list = list(zip(field_names, operands, values))
    print(critera_list)
    

    c = conn.cursor()
    sql = """ SELECT * FROM StooqCrossTabTable WHERE """
    criteria_no = 0
    
    if (len(field_names) > 1):
        multiple_criteria = True
        
    for criteria in critera_list:
        criteria_no = criteria_no + 1
        
        sql = sql + "( [" + criteria[0] +"] " + criteria[1] + " "
        field_type = getFieldType(criteria[0], column_details)
    
#    print(field_type)
# result = zip(numberList, strList)
    
        if (field_type ==  str):
            sql = sql + "'" + criteria[2] + "' )"

        if (field_type ==  datetime.datetime):
            sql = sql + criteria[2] + " )"
                                  
        if (multiple_criteria and criteria_no < len(field_names)):
            sql = sql + " AND "
        
    print(sql)
    c.execute(sql)

    columns = [column[0] for column in c.description]
    
    recs = []
    for row in c.fetchall():
        recs.append(dict(zip(columns, row)))
    
    return recs

if __name__ == '__main__': 
    db_file = r'''StooqDataAnalysis.accdb'''  #raw string, escape sequences are ignored
    db_file = os.path.abspath(db_file)
    dbConnData = dbu.createDBConnection(db_file)
    conn = dbu.createDBConnection(db_file)   
    
# Selective records ===========================================================
#     field_names = ['Ticker', 'CurDate']
#     operands = ['=', '=']
# #    values = ['A','datetime.date(year=2018, month=3, day=15)']
#     values = ['A','#3/16/2018#']
#     
#     in_recs = get_selective_recs(conn, field_names, operands, values)
#     
#     for rec in in_recs:
#         for k, v in rec.items():
#              print(k, v)
#         break
#===============================================================================
    
# datetime.time(hour=4, minute=0, second=0)    
#in_recs = get_all_recs(conn)

#===============================================================================
# for rec in in_recs:
#     for k, v in rec.items():
#         print(k, v)
#     break
#===============================================================================
        

    

## db_utils ##

In [ ]:
'''
Program Name: db_utils

Created on Jun 7, 2018

@author: karsu
'''

import pyodbc
#import datetime
#import sys

def createDBConnection(db_file):
    user = 'admin'
    password = ''   
    odbc_conn_str = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=%s;UID=%s;PWD=%s' %\
                (db_file, user, password)    #string variable formatted according to pyodbc specs
                
    try:
        conn = pyodbc.connect(odbc_conn_str)
    #            print("connection made")
    except Exception as e:
        print('Database connection is failed')
        print(e)               
#    c = conn.cursor()
#    list1 = [conn, c]
    return conn

def getTableColumns(conn, table_name):
    c = conn.cursor()    
    sql = """ SELECT * FROM """
    sql = sql + table_name
    c.execute(sql)
    columns = [column[0] for column in c.description]
    return columns


def getTableColumnsDetails(conn, table_name):
    c = conn.cursor()    
    sql = """ SELECT * FROM """
    sql = sql + table_name
    c.execute(sql)
    return c.description


def getAllTablesInDB(conn):
    c = conn.cursor()
    tables = [table for table in c.tables() if table[3] == 'TABLE']    
    return tables

def getAllViewsInDB(conn):
    c = conn.cursor()
    views = [table for table in c.tables() if table[3] == 'VIEW']    
    return views

def get_all_recs(conn, tablename):
    
    c = conn.cursor()
    sql = """ SELECT * FROM """
    sql = sql + tablename
    c.execute(sql)

    columns = [column[0] for column in c.description]
    
    recs = []
    for row in c.fetchall():
        recs.append(dict(zip(columns, row)))
    
    return recs

if __name__ == '__main__': 
    pass


## Plotting Candle Stick Charts ##

Now that we have transposed data for 6 time slots, the next step is to plot the candle stick chart  
for a ticker-date combination, plotting 6 candles corresponding to those 6 time slots. 

It visually gives a better idea of the gap's progress during the morning session.

### View Objects ###

There are 3 python programs written to accomplish this. 

The first one, **chartDisplay.py**, reads the transposed MS Access table, **5MtsBaseForAnalysisSorted**,  
sets up the GUI front end, display the Ticker/Date in a Treeview control. On selection of a Ticker/Date  
row in treeview, it calls a function,  
+    **get_selected_cross_tab_rec(ticker_selected, date_selected)**  
from **read_selected_stooq_cross_tab.py** program. This function gathers candle stick data  
(open, low, high, close) for all 6 ticks and calls a function,  
+ **plot_chart(times, opens, highs, lows, closes, file_name)**  
from **plotCSChartSingle.py** program. This function calls a **plotly**, a popular  
data visualization library passing all the necessary data and saves the output Candle Stick chart **cs2.png**.  

Then **chartDisplay.py** displays the chart in a label control, under the treeview control.

### chartDisplay.py ###

In [4]:
'''
Program Name - chartDisplay.py

Created on Jun 14, 2018

@author: karsu
'''
from tkinter import *
from tkinter import ttk
import datetime
import io_5MtsBaseForAnalysisSorted as base5
import io_StooqCrossTabTable as sct
import db_utils as dbu
import plotCSChartSingle as plot
import read_selected_stooq_cross_tab as rct
import os


class Application(Frame):
    def __init__(self, master):
        ttk.Frame.__init__(self, master)
        root.minsize(width=600, height=800)
        root.resizable(width=0, height=0)
        self.grid()
        self.create_widgets()
    
    
    def create_widgets(self):
#        root = Tk()

        self.image = PhotoImage(file="cs1.png")
        self.label1 = ttk.Label(image=self.image)
        self.label1.grid(row=22, column = 0, columnspan = 8, padx=5, pady = 5)

        self.tv = ttk.Treeview(root,selectmode='browse')
        self.tv = ttk.Treeview(self, height=20)
        self.tv.place(x=30, y=95)

        self.vsb = ttk.Scrollbar(root, orient="vertical", command=self.tv.yview)
        self.vsb.place(x=30+656+2, y=10, height=400+20)
        
        self.tv.configure(yscrollcommand=self.vsb.set)

        self.tv['columns'] = ('Ticker', 'Tr Date')
        self.tv.heading("#0",text="Time", anchor = "w")
        self.tv.column('#0', stretch=NO, width=5, anchor='w')
        self.tv.heading('Ticker', text='Ticker')
        self.tv.column('Ticker', anchor='center', width = 80)
        self.tv.heading('Tr Date', text='Tr Date')
        self.tv.column('Tr Date', anchor='center', width = 612)
        self.tv.bind('<ButtonRelease-1>', self.select_item)
        self.tv.grid(row=1, column = 0, columnspan = 30, padx=5, pady = 5)
        self.treeview = self.tv
        
        ttk.Style().configure("Treeview", font = ('',11), background="#383838#",
        foreground="white", fieldbackground = "yellow")

        db_file = 'StooqDataAnalysis.accdb'  
        db_file = os.path.abspath(db_file)
        conn = dbu.createDBConnection(db_file)

        in_recs = sct.get_all_StooqCrossTabTable_recs(conn)
        print(len(in_recs))
        
        for rec in in_recs:
#            print(str(rec['CurDate']))
            datestr = str(rec['CurDate'])[:10]
#            print(datestr)
            year,month,day = str(datestr).split("-")
#            print(year,month,day)
            ticker1 = rec['Ticker']
            self.tv.insert("", "end",  values = (ticker1, datetime.date(year = int(year), month=int(month), day=int(day))))

             
    def select_item(self, a):
        test_str_library = self.tv.item(self.tv.selection())  # gets all the values of selected row
#        print('The test str = ', type(test_str_library), test_str_library, '\n') #prints a dictionary of the selected row
        item = self.tv.selection()[0] # which row did you click on
#        print('item clicked', item) # variable that represents the row you clicked on
        ticker_selected = self.tv.item(item)['values'][0] # prints the first value of the values list
        date_selected = self.tv.item(item)['values'][1]
#        print(ticker_selected, date_selected)

        myfile="cs2.png"
        if os.path.isfile(myfile):
            os.remove(myfile)
            print('cs2.png removed)')

        rct.get_selected_cross_tab_rec(ticker_selected, date_selected)

        self.image2 = PhotoImage(file=myfile)
#        self.label1.image = image
#        self.label1.grid(row=22, column = 0, columnspan = 8, padx=5, pady = 5)
        self.label1.configure(image=self.image2)
        self.label1.image=self.image2

        
root = Tk()
Application(root)
root.mainloop()

32
cs2.png removed)
[('Ticker', '=', 'AAN'), ('CurDate', '=', '#03/16/2018#')]
 SELECT * FROM StooqCrossTabTable WHERE ( [Ticker] = 'AAN' ) AND ( [CurDate] = #03/16/2018# )


### read_selected_stooq_cross_tab.py ###

In [ ]:
'''
Program Nmae - read_selected_stooq_cross_tab.py

Created on Jun 15, 2018

@author: karsu
'''
import os, sys
import pyodbc
import makeAccessDBConnection
import datetime
import logging
import io_5MtsBaseForAnalysisSorted as base5
import io_StooqCrossTabTable as sct
import db_utils as dbu
import plotly.plotly as plotly
import plotly.graph_objs as plotlygo
import plotCSChartSingle

def get_selected_cross_tab_rec(ticker, date):
    
#    print(ticker, date)
    year,month,date = date.split("-")
#    print(year, month, date)
    date = "#" + month + "/" + date + "/" + year + "#"
#    db_file = r'''C:\TickData2018\StooqDataAnalysis.accdb'''  #raw string, escape sequences are ignored
    db_file = r'''StooqDataAnalysis.accdb'''  #raw string, escape sequences are ignored
    db_file = os.path.abspath(db_file)
    conn = dbu.createDBConnection(db_file)
    
    field_names = ['Ticker', 'CurDate']
    operands = ['=', '=']
    #    values = ['A','datetime.date(year=2018, month=3, day=15)']
    #values = ['A','#3/16/2018#']
    values = [ticker,date]
    
    in_recs = sct.get_selective_recs(conn, field_names, operands, values)
    
    for rec in in_recs:
 
        times = [datetime.time(hour=4, minute=0, second=0),
             datetime.time(hour=9, minute=35, second=0),
             datetime.time(hour=10, minute=0, second=0),
             datetime.time(hour=10, minute=30, second=0),
             datetime.time(hour=11, minute=0, second=0),
             datetime.time(hour=16, minute=0, second=0)]
         
        opens = [rec['PDayOpen'],
                 rec['CDayOpenOpen'],
                 rec['CDay1000AMOpen'],
                 rec['CDay1030AMOpen'],
                 rec['CDay1100AMOpen'],
                 rec['CDay0400PMOpen']
                 ]
        
        lows = [rec['PDayLow'],
                 rec['CDayOpenLow'],
                 rec['CDay1000AMLow'],
                 rec['CDay1030AMLow'],
                 rec['CDay1100AMLow'],
                 rec['CDay0400PMLow'],
                ]
    
        highs = [rec['PDayHigh'],
                 rec['CDayOpenHigh'],
                 rec['CDay1000AMHigh'],
                 rec['CDay1030AMHigh'],
                 rec['CDay1100AMHigh'],
                 rec['CDay0400PMHigh'],
                ]
                 
        closes = [rec['PDayClose'],
                 rec['CDayOpenClose'],
                 rec['CDay1000AMClose'],
                 rec['CDay1030AMClose'],
                 rec['CDay1100AMClose'],
                 rec['CDay0400PMClose'],
                ]
                             
        file_name = 'cs2'
        plotCSChartSingle.plot_chart(times, opens, highs, lows, closes, file_name)
    
#    print(len(in_recs))
    
if __name__ == '__main__':
    date = "2018-03-15" 
    get_selected_cross_tab_rec('A', date)


### plotCSChartSingle.py ###

In [ ]:
'''
Program Name - plotCSChartSingle.py

Created on Jun 14, 2018

@author: karsu
'''
import os, sys
import pyodbc
import makeAccessDBConnection
import datetime
import logging
import io_5MtsBaseForAnalysisSorted as base5
import io_StooqCrossTabTable as sct
import db_utils as dbu
import plotly.plotly as plotly
import plotly.graph_objs as plotlygo

def plot_chart(times, opens, highs, lows, closes, file_name):
    
    trace = plotlygo.Candlestick(x=times,
                       open=opens,
                       high=highs,
                       low=lows,
                       close=closes)
    
    layout = plotlygo.Layout(
        xaxis = dict(
            rangeslider = dict(
                visible = False
            )
        )
    )
    data = [trace]
    fig = plotlygo.Figure(data=data,layout=layout)
    plotly.image.save_as(fig, filename= file_name +'.png')